##### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2022 Semester 1

## Assignment 2: Sentiment Classification of Tweets

**Split the training data**

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

def data_split(filename, test_size):
    df = pd.read_csv(filename, sep=',', skipinitialspace=True)
    X = df['text']
    y = df['sentiment']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=False)
    
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = data_split("Train.csv", 0.1)

**Feature vectorization**

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

**Classification**

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB


lgr = LogisticRegression(max_iter=200)
lgr.fit(X_train_tfidf, y_train)

def evaluate(X_train, y_train, X_test, y_test, model):
    model.fit(X_train, y_train)
    print("Accuracy:", model.score(X_test, y_test))

Accuracy: 0.6519944979367263
